# Collaborative Filtering

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Introduction

# Data

In [2]:
tags = pd.read_csv('datasets/ml-latest-small/tags.csv')
ratings = pd.read_csv('datasets/ml-latest-small/ratings.csv')
movies = pd.read_csv('datasets/ml-latest-small/movies.csv')
links = pd.read_csv('datasets/ml-latest-small/links.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
df = movies.merge(ratings, on = 'movieId')
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [8]:
userRatings = df.pivot_table(index = ['userId'], columns = ['title'],
                            values = 'rating')
userRatings.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Similarity Metric

## Pearson Similarity

The Pearson Similarity metric is used to measure the rating vectors of two users. We will denote the users as user *1* and user *2*. Additionally, *I_1* and *I_2* will denote the set of items user *1* and user *2* has rated respectfully.

The first step to computing the Pearson Similarity is computing the mean rating for each user. The mean rating of user *1* is computed with the following equation:

$$\mu_1 = \frac{\sum_i r_{1,i}}{|I_1|}$$

where *i* is the index of the item therefore $r_{1,i}$ is the rating user *1* gave on item *i*

The pearson similarity can then be computed between the two users:

$$Pearson(1,2) = \frac{\sum_i ((r_{1,i}-\mu_1)(r_{2,i} - \mu_2))}{\sqrt{\sum_i ((r_{1,i}-\mu_1)^2}\sqrt{\sum_i ((r_{2,i}-\mu_2)^2}}$$

The Pearson Similarity is computed between a target user and all the other users. We can then find the *k* number of users with the highest Pearson Similarity with the target user.

### Predicting Ratings
Once we have the *k* users most similar to the target user, we can use the Pearson Similarity scores and the user-item ratings of the similar users to predict the score that target user would give on the items they have not rated. Afterwards you would just recommend the top items based off the rated scores. 

For example, lets say user 1 was our target user and user 2 and 3 were the most similar users. User 1 has not rated items 4 and 5, but user 2 and 3 have. We would predict the rating user 1 would give to items 4 and 5 with the following functions:

$$r_{1,4} = \frac{r_{2,4}*pearson(1,2) + r_{3,4}*pearson(1,3)}{pearson(1,2) + pearson(1,3)}$$

### Mean-Centered Ratings

The problem with the above metric is that it does not take into account that users may rate things differently. For example, user *u* might be very lenient with their ratings and rate things highly whereas user *v* is a tough critique and rarely gives out high reviews. The ratings need to be mean-centered before predicting ratings (the pearson similarity remains the same). To compute the mean-centered rating (we will denote this as $s_{ui}$)for user *u* on item *i* you would simply substract the rating given to item *i* by user *u* with the average rating of user *u*.

$$s_{ui} = r_{ui} - \mu_u$$

# Evaluation Metric

### Root Mean Square Error (RMSE)

# Baseline Model

# Item-Based Collaborative Filtering

## Advantages and Disadvantages

The advantage of Item-Based Collaborative Filtering is that it often provides more relevant recommendations because it using your OWN ratings to make recommendations. For example a recommonder system might look at amovie you've enjoyed and rated highly and recommend similar movies.

Item-Based ratings are also more stable to changes in ratings. This is because for User-Based ratings, there are a lot more users than items. This means that there will be cases where two users have a small number of the same items, but two items are much more likely to have a larger number of users who have rated both of them. This means that for User-Based ratings, just adding a few ratings can change the similarity score a lot, whereas for Item-Based it is much more stable to additions of new ratings.

The disadvantage of Item-Based Collaborative Filtering is that they may not provide more diverse recommendations as oppose to User-Based Collaborative Filtering. Recommending more diverse items may lead to pleasant surprises or new found interests. Without enough diversity, it is possible that a user can get border with similar recommendations to items they've been recommended.